In [94]:
import matplotlib.pyplot as plt
import numpy as np
from numpy.linalg import norm
from scipy.integrate import odeint
import pprint
from ipywidgets import interact, interactive, fixed, interact_manual, jslink
import ipywidgets as widgets
import ipyvolume as ipv
import ipyvolume.pylab as p3
import traitlets
import time, sys
from IPython.core.display import clear_output
from random import random

In [2]:
# define the atoms in the chain (this test will only use a 3 atom chain)
#r1 = (0, 0, 0)
#r2 = (0, 1, 0)
#r3 = (2, 3, 0)
#r4 = (4, 1, 0)

#floaters = np.hstack((r1, r2, r3, r4))

#m = 4             # total atoms in the chain

In [104]:
# define the number of floating atoms
floaters = np.zeros(4*3) # set number of floaters and multiply by 3 to have 3 coords for each
n = len(floaters)
m = n/3                    # total atoms in the chain

# set the positions of the floating atoms
count = 0
k = 1.7
for i in range(0,n,3):
    floaters[i] = random()
    floaters[i+1] = random()
    floaters[i+2] = random()
    count += 1

In [112]:
# define parameters
beta = 200

In [138]:
# define time interval
t = np.linspace(0,10,50)

In [133]:
# bending force DE system
def bending_force(floaters, t):
    
    current_bending_force = np.zeros((int(m),3))
    total_bending_force = np.zeros(int(m))
    
    # loop for computing bending force
    for i in range(1,int(m-1)):          # loop through each atom not on the end of the chain and compute the bending force on it and its neighbors
        
        # define the pieces of the bending energy equation
        sx_n = floaters[i*3] - floaters[(i-1)*3]
        sx_n1 = floaters[(i+1)*3] - floaters[i*3]

        sy_n = floaters[i*3 + 1] - floaters[(i-1)*3 + 1]
        sy_n1 = floaters[(i+1)*3 + 1] - floaters[i*3 + 1]

        sz_n = floaters[i*3 + 2] - floaters[(i-1)*3 + 2]
        sz_n1 = floaters[(i+1)*3 + 2] - floaters[i*3 + 2]

        A = norm((sx_n, sy_n, sz_n))*norm((sx_n1, sy_n1, sz_n1))

        B = np.dot((sx_n, sy_n, sz_n),(sx_n1, sy_n1, sz_n1))

        partial_common = .5/A
        
        # partials of A wrt x for atom n, n-1 and n+1, respectively
        Axn = partial_common*(2*(sx_n)*(sx_n1**2) - 2*(sx_n1)*(sx_n**2) + 2*(sx_n)*(sy_n1**2) + 2*(sx_n)*(sz_n1**2) - 2*(sx_n1)*(sy_n**2) - 2*(sx_n1)*(sz_n**2))
        Axn0 = partial_common*(-2*(sx_n)*(sx_n1**2) - 2*(sx_n)*(sy_n1**2) - 2*(sx_n)*(sz_n1**2))
        Axn1 = partial_common*(2*(sx_n**2)*(sx_n1) + 2*(sy_n**2)*(sx_n1) + 2*(sz_n**2)*(sx_n1))
        
        # partials of A wrt y for atom n, n-1 and n+1, respectively
        Ayn = partial_common*(-2*(sx_n**2)*(sy_n1) + 2*(sy_n)*(sx_n1**2) + 2*(sy_n)*(sy_n1**2) - 2*(sy_n**2)*(sy_n1) + 2*(sy_n)*(sz_n1**2) - 2*(sz_n**2)*(sy_n1))
        Ayn0 = partial_common*(-2*(sy_n)*(sx_n1**2) - 2*(sy_n)*(sy_n1**2) - 2*(sy_n)*(sz_n1**2))
        Ayn1 = partial_common*(2*(sy_n**2)*(sy_n1) + 2*(sz_n**2)*(sy_n1) + 2*(sx_n**2)*(sy_n1))
        
        # partials of A wrt z for atom n, n-1 and n+1, respectively
        Azn = partial_common*(-2*(sx_n**2)*(sz_n1) - 2*(sy_n**2)*(sz_n1) - 2*(sz_n**2)*(sz_n1) + 2*(sz_n)*(sz_n1**2) + 2*(sz_n)*(sx_n1**2) + 2*(sz_n)*(sy_n1**2))
        Azn0 = partial_common*(-2*(sz_n)*(sx_n1**2) -2*(sz_n)*(sy_n1**2) - 2*(sz_n)*(sz_n1**2))
        Azn1 = partial_common*(2*(sz_n1)*(sx_n**2) + 2*(sz_n1)*(sy_n**2) + 2*(sz_n1)*(sz_n**2))
        
        # partials of B wrt x for atom n, n-1 and n+1, respectively
        Bxn = floaters[(i+1)*3] - 2*floaters[i*3] + floaters[(i-1)*3]
        Bxn0 = -floaters[(i+1)*3] + floaters[i*3]
        Bxn1 = floaters[i*3] - floaters[(i-1)*3]

        # partials of B wrt y for atom n, n-1 and n+1, respectively
        Byn = floaters[(i+1)*3 + 1] - 2*floaters[i*3 + 1] + floaters[(i-1)*3 + 1]
        Byn0 = -floaters[(i+1)*3 + 1] + floaters[i*3 + 1]
        Byn1 = floaters[i*3 + 1] - floaters[(i-1)*3 + 1]
        
        # partials of B wrt z for atom n, n-1 and n+1, respectively
        Bzn = floaters[(i+1)*3 + 2] - 2*floaters[i*3 + 2] + floaters[(i-1)*3 + 2]
        Bzn0 = -floaters[(i+1)*3 + 2] + floaters[i*3 + 2]
        Bzn1 = floaters[i*3 + 2] - floaters[(i-1)*3 + 2]
        
        try:
            Eb_common = 4*beta/((A+B)**2)                   # compute common part of bending energy gradient
            if (np.isnan(Eb_common)  or np.isinf(Eb_common) or np.isnan(A)):
                raise ValueError('Divide by zero')
                raise Exception
            
        except Exception as error:
            print("Caught exception: ", error)
            continue
            
        current_bending_force[i,0] += Eb_common*(A*Bxn - B*Axn)
        current_bending_force[i,1] += Eb_common*(A*Byn - B*Ayn)
        current_bending_force[i,2] += Eb_common*(A*Bzn - B*Azn)
            
        current_bending_force[i-1,0] += Eb_common*(A*Bxn0 - B*Axn0)
        current_bending_force[i-1,1] += Eb_common*(A*Byn0 - B*Ayn0)
        current_bending_force[i-1,2] += Eb_common*(A*Bzn0 - B*Azn0)
            
        current_bending_force[i+1,0] = Eb_common*(A*Bxn1 - B*Axn1)
        current_bending_force[i+1,1] = Eb_common*(A*Byn1 - B*Ayn1)
        current_bending_force[i+1,2] = Eb_common*(A*Bzn1 - B*Azn1)

    total_bending_force = np.hstack(current_bending_force)
    return total_bending_force

In [134]:
%%time

# solve the system of ODEs to compute changing positions as a result of the bending force acting on the chain
gFlow = odeint(bending_force, floaters, t, atol=1.4e-10)

CPU times: user 185 ms, sys: 11.6 ms, total: 197 ms
Wall time: 188 ms


/anaconda/lib/python3.6/site-packages/scipy/integrate/odepack.py:218: ODEintWarning: Excess work done on this call (perhaps wrong Dfun type). Run with full_output = 1 to get quantitative information.
  warnings.warn(warning_msg, ODEintWarning)


In [135]:
print(gFlow)

[[  9.20500708e-002   4.14341921e-001   7.22127963e-001 ...,
    6.88665862e-001   7.48626025e-001   6.58507938e-002]
 [  1.22168436e-002   5.20394332e-002   8.59317177e-001 ...,
    1.04849439e+000   6.48794447e-001   1.17050732e-001]
 [  6.01428955e+175   1.11317714e+171   4.19314256e+228 ...,
    7.20870378e+159   2.43861180e-152   1.32882766e+179]
 ..., 
 [  0.00000000e+000   6.36600002e-314   0.00000000e+000 ...,
    0.00000000e+000   1.69759820e-313   0.00000000e+000]
 [  0.00000000e+000   6.36600002e-314   0.00000000e+000 ...,
    0.00000000e+000   3.03553933e-320   0.00000000e+000]
 [  0.00000000e+000   6.36600002e-314   0.00000000e+000 ...,
    0.00000000e+000   2.12199736e-313   0.00000000e+000]]


## Create Animation to Check that The Force is Correct

In [136]:
# define arrays of x, y and z coordinates to create an animated scatter plot
x = np.zeros((len(t),int(m)))
y = np.zeros((len(t),int(m)))
z = np.zeros((len(t),int(m)))
for i in range(len(t)):
    for j in range(int(m)):
        x[i,j] = gFlow[i,j*3]
        y[i,j] = gFlow[i,j*3+1]
        z[i,j] = gFlow[i,j*3+2]

In [137]:
%%time
%matplotlib inline

# create an ipyvolume.pylab figure for the simulation
p3.figure()

# plot a scatterplot of the arrays defined above in the figure
s1 = p3.scatter(x, y, z, marker='sphere', connected=True)
s2 = p3.plot(x, y, z)

# set the x, y and z limits of the figure
p3.xlim(-10,10)
p3.ylim(-10,10)
p3.zlim(0,2)

# add animation controls to the figure
p3.animation_control([s1, s2], interval=400)

# add sliders for parameters
beta_slider = widgets.IntSlider(min=0, max=15, step=1, value=beta, description='beta', continuous_update=False)

# create handler functions to change the values of the parameters globally when the sliders change
def on_beta_slider_change(change):
    global beta
    beta = change.new

# set the sliders to call their handler functions any time the sliders are moved
beta_slider.observe(on_beta_slider_change, names='value')

# display the VBox beneath the total system energy plot
label = widgets.Label('Parameter sliders:')
parameters = widgets.VBox([ipv.gcc(), label, beta_slider])
display(parameters)

VBox(children=(VBox(children=(Figure(animation=400.0, animation_exponent=1.0, camera_center=[0.0, 0.0, 0.0], height=500, matrix_projection=[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], matrix_world=[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], scatters=[Scatter(color_selected=array('white',
      dtype='<U5'), connected=True, geo='sphere', size=array(2), size_selected=array(2.6), x=array([[  9.20500708e-002,   3.39521804e-001,   8.07296643e-001,
          6.88665862e-001],
       [  1.22168436e-002,   4.08785757e-001,   4.58037394e-001,
          1.04849439e+000],
       [  6.01428955e+175,   5.47820448e+241,   6.01347002e-154,
          7.20870378e+159],
       ..., 
       [  0.00000000e+000,   0.00000000e+000,   0.00000000e+000,
          0.00000000e+000],
       [  0.00000000e+000,   0.00000000e+000,   0.00000000e+000,
          0.00000000e+000],
       [  0.00000000e+000,   0.00000000e+000,   0.00000000e+000,
          0.00000000e+000]]), y=array([[  4.14341921e-001,   7.91221982e-002,   4.79151970e-002,
          7.48626025e-001],
       [  5.20394332e-002,   2.80404654e-001,   3.08766808e-001,
          6.48794447e-001],
       [  1.11317714e+171,   4.21476530e+198,   5.95505258e+228,
          2.43861180e-152],
       ..., 
       [  6.36600002e-314,   1.06099946e-313,   2.52961611e-320,
          1.69759820e-313],
       [  6.36600002e-314,   2.40313530e-320,   2.52961611e-320,
          3.03553933e-320],
       [  6.36600002e-314,   1.06099946e-313,   2.52961611e-320,
          2.12199736e-313]]), z=array([[  7.22127963e-001,   4.48493257e-001,   8.55148085e-001,
          6.58507938e-002],
       [  8.59317177e-001,   5.75265083e-001,   5.39987106e-001,
          1.17050732e-001],
       [  4.19314256e+228,   2.09505757e-110,   2.38763339e+190,
          1.32882766e+179],
       ..., 
       [  0.00000000e+000,   0.00000000e+000,   0.00000000e+000,
          0.00000000e+000],
       [  0.00000000e+000,   0.00000000e+000,   0.00000000e+000,
          0.00000000e+000],
       [  0.00000000e+000,   0.00000000e+000,   0.00000000e+000,
          0.00000000e+000]])), Scatter(color_selected=None, connected=True, size=array(1), size_selected=array(1), visible_lines=True, visible_markers=False, x=array([[  9.20500708e-002,   3.39521804e-001,   8.07296643e-001,
          6.88665862e-001],
       [  1.22168436e-002,   4.08785757e-001,   4.58037394e-001,
          1.04849439e+000],
       [  6.01428955e+175,   5.47820448e+241,   6.01347002e-154,
          7.20870378e+159],
       ..., 
       [  0.00000000e+000,   0.00000000e+000,   0.00000000e+000,
          0.00000000e+000],
       [  0.00000000e+000,   0.00000000e+000,   0.00000000e+000,
          0.00000000e+000],
       [  0.00000000e+000,   0.00000000e+000,   0.00000000e+000,
          0.00000000e+000]]), y=array([[  4.14341921e-001,   7.91221982e-002,   4.79151970e-002,
          7.48626025e-001],
       [  5.20394332e-002,   2.80404654e-001,   3.08766808e-001,
          6.48794447e-001],
       [  1.11317714e+171,   4.21476530e+198,   5.95505258e+228,
          2.43861180e-152],
       ..., 
       [  6.36600002e-314,   1.06099946e-313,   2.52961611e-320,
          1.69759820e-313],
       [  6.36600002e-314,   2.40313530e-320,   2.52961611e-320,
          3.03553933e-320],
       [  6.36600002e-314,   1.06099946e-313,   2.52961611e-320,
          2.12199736e-313]]), z=array([[  7.22127963e-001,   4.48493257e-001,   8.55148085e-001,
          6.58507938e-002],
       [  8.59317177e-001,   5.75265083e-001,   5.39987106e-001,
          1.17050732e-001],
       [  4.19314256e+228,   2.09505757e-110,   2.38763339e+190,
          1.32882766e+179],
       ..., 
       [  0.00000000e+000,   0.00000000e+000,   0.00000000e+000,
          0.00000000e+000],
       [  0.00000000e+000,   0.00000000e+000,   0.00000000e+000,
          0.00000000e+000],
       [  0.00000000e+000,   0.00000000e+000,   0.00000000e+00

CPU times: user 395 ms, sys: 97.2 ms, total: 493 ms
Wall time: 391 ms
